# This project implemented by Elham Mahdipour
## She is a Ph.D. Candidate of computer engineering at Yazd University, Yazd, Iran.
## Please feel free and contact to me: elham.mahdipour@gmail.com/ elham.mahdipour@stu.yazd.ac.ir

# Phase 1 
## Create Dataset and Preprocessing

In [1]:
import networkx as nx
G1=nx.read_weighted_edgelist('large dataset\sc-sc.evals')
G1

In [2]:
G2=nx.read_weighted_edgelist('large dataset\ec-ec.evals')
G2

In [3]:
### Check and Swap if G1 > G2 ###
if len(G1)>len(G2):
    temp=G1
    G1=G2
    G2=temp
print(len(G1))
print(len(G2))

4265
6613


In [4]:
G_target_na=nx.read_weighted_edgelist('large dataset\ec-sc.evals')
G_target_na  

In [5]:
ed1=G1.edges()
ed2=G2.edges()

nd1=G1.nodes()
nd2=G2.nodes()

el1=list(ed1)
el2=list(ed2)

nd1=list(nd1)
nd2=list(nd2)

degG1 = [val for (node, val) in G1.degree()]
degG2 = [val for (node, val) in G2.degree()]

# Feature Extraction

# compute score for create similarity matrix

In [6]:
def deg_Diff(G1,G2):
    Degree_Difference=np.zeros((len(G1),len(G2)))
    for i in range(len(G1)):
        for j in range(len(G2)):
            Degree_Difference[i][j]=abs(degG1[i]-degG2[j])/max(degG1[i],degG2[j])
    return Degree_Difference

In [7]:
def score_pageRank(X):
    a=nx.pagerank(X)
    return a

In [8]:
def coefficient_pagerank(x,y):  #x is G1, y is G2
   # print(len(x))
    p1=score_pageRank(x)
    b=p1.values()
    pr1=list(b)
    p2=score_pageRank(y)
    c=p2.values()
    pr2=list(c)
    pr=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            #print(pr1[i],pr2[j])
            pr[i][j]=abs(pr1[i]-pr2[j])/max(pr1[i],pr2[j])   #minimum pr is maximum similarity of topology 
    return pr

In [9]:
def coefficient_edges(index_node, G, GraphNumber):
    if GraphNumber==1:
        sum_edge=0        
        for i in G.neighbors(nd1[index_node]):                       
            sum_edge=sum_edge+degG1[nd1.index(i)]
        #print(sum_edge)
        temp=(degG1[index_node]-1) if degG1[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG1[index_node]*temp)
    else:
        sum_edge=0        
        for i in G.neighbors(nd2[index_node]):                       
            sum_edge=sum_edge+degG2[nd2.index(i)]
        #print(sum_edge)
        temp=(degG2[index_node]-1) if degG2[index_node]> 1 else 1        
        coeff_node=(2*sum_edge)/(degG2[index_node]*temp)
    return coeff_node

In [10]:
def compute_Ea(G1,G2):
    Ea_G1=np.zeros(len(G1))
    Ea_G2=np.zeros(len(G2))
    for i in range(len(G1)):
        Ea_G1[i]=coefficient_edges(i, G1, 1)
    for j in range(len(G2)):
        Ea_G2[j]=coefficient_edges(j,G2,2)
    ea=[Ea_G1, Ea_G2]
    return(ea)

In [11]:
# compute relative clustering coefficient difference between node a (in G1) and node b (in G2)
def CD(G1, G2):
    cd=np.zeros((len(G1),len(G2)))
    EA=compute_Ea(G1,G2)
    #print(EA[0])      #Ea for G1
    #print("===================")
    #print(EA[1])      #Ea for G2
    for i in range(len(G1)):
        for j in range(len(G2)):
            cd[i,j]=abs(EA[0][i]-EA[1][j])/max(EA[0][i],EA[1][j])
    return cd

In [12]:
def sequence_score(x,y):
    seq=np.zeros((len(x),len(y)))
    for i in range(len(x)):
        for j in range(len(y)):
            q1=G_target_na.get_edge_data(str(nd1[i]),str(nd2[j]))
            if q1==None:
                c=0
            else:
                c=list(q1.values())
                c=c[0]
            seq[i][j]=c    
            
    return seq

In [13]:
def compute_score(G1,G2):
    coeff_pr=coefficient_pagerank(G1,G2)
    dd=deg_Diff(G1,G2)
    cd=CD(G1,G2)
    seq_sc=sequence_score(G1,G2)
    
    alpha=0.1
    betta=0.2
    gamma=0.2
    zetta=1-alpha-betta-gamma
    s=alpha*(1-coeff_pr)+betta*(1-dd)+gamma*(1-cd)+zetta*seq_sc
    return s,coeff_pr, dd, cd

In [14]:
import numpy as np
m, coeff_pr, dd, cd=compute_score(G1,G2)
sim=m

# Phase 2: Dataset Generation

# create data for deep learning

# change problem to classification 
## [node of G1, node of G2, Coefficient page rank, clustering coefficient difference,  similarity score, alignment=yes(1) or no(0)]

In [15]:
import numpy as np
int_nd1=np.zeros(len(nd1))
int_nd2=np.zeros(len(nd2))

species=['ec','sc','ce','dm','mm','hs']
ch1=0  #please select index for first species of species list, for example index of ec is 0
ch2=1  #please select index for second species of species list, for example index of sc is 1
for i in range(len(nd1)):
    if (species[ch1] in nd1[i] or species[ch2] in nd1[i]):
        s=nd1[i][2:]
        x=int(s)
        int_nd1[i]=x    
for i in range(len(nd2)):
    if (species[ch1] in nd2[i] or species[ch2] in nd2[i]):
        s=nd2[i][2:]
        x=int(s)
        int_nd2[i]=x    

In [16]:
# en_mat is encoding matrix
en_mat=[]

for i in range(len(nd1)):
    for j in range(len(nd2)):
        if G_target_na.has_edge(nd1[i],nd2[j]):
            align_class='Yes'
        else:
            align_class='No'
        
        sample=[int_nd1[i],int_nd2[j], coeff_pr[i][j], dd[i][j],cd[i][j],sim[i][j],align_class] 
        en_mat.append(sample)
print(len(en_mat))

28204445


In [17]:
yc=[]
noc=[]
for i in range(len(en_mat)):
    if en_mat[i][6]=='Yes':
        yc.append(en_mat[i])
    else:
        noc.append(en_mat[i])
print(len(yc), len(noc))

4561 28199884


In [18]:
data=yc+noc 
len(data)
X=[]
y=[]
for i in range(len(data)):
    X.append(data[i][0:6])
    y.append(data[i][6])

len(X),len(y)

(28204445, 28204445)

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [20]:
import pickle
with open('large dataset/ec-sc-data.pickle', 'wb') as f:
    pickle.dump([X_train, y_train,X_test,y_test],f)

In [1]:
import pickle
with open('large dataset/ec-sc-data.pickle','rb') as f:
    X_train, y_train,X_test,y_test=pickle.load(f)

In [21]:
print(len(X_train), len(y_train), len(X_test), len(y_test))

25384000 25384000 2820445 2820445


In [22]:
x_tr,y_tr,x_te,y_te=X_train, y_train,X_test,y_test

# Solve data Imbalance problem

# Resample 

In [23]:
from collections import Counter
from imblearn.combine import SMOTEENN

print('Original dataset shape %s' % Counter(y_train))
#sampling_strategy = 'not minority'
smt = SMOTEENN(random_state=42) #sampling_strategy=sampling_strategy)

X_res_train, y_res_train = smt.fit_resample(X_train, y_train)
print('Resampled dataset shape %s' % Counter(y_res_train))   

Using TensorFlow backend.
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

Original dataset shape Counter({'No': 25379873, 'Yes': 4127})
Resampled dataset shape Counter({'No': 25379873, 'Yes': 25379873})


In [24]:
print('Original dataset shape %s' % Counter(y_test))
smt = SMOTEENN(random_state=42)
X_res_test, y_res_test = smt.fit_resample(X_test, y_test)
print('Resampled dataset shape %s' % Counter(y_res_test))  

Original dataset shape Counter({'No': 2820011, 'Yes': 434})
Resampled dataset shape Counter({'No': 2820011, 'Yes': 2820011})


In [25]:
import numpy as np
X_res_train=np.array(X_res_train)
y_res_train=np.array(y_res_train)
X_res_test=np.array(X_res_test)
y_res_test=np.array(y_res_test)

In [26]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_res_train)
encoded_Y = encoder.transform(y_res_train)
# One Hot Encode
y_res_train = np_utils.to_categorical(encoded_Y)

In [27]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_res_test)
encoded_Y = encoder.transform(y_res_test)
# One Hot Encode
y_res_test = np_utils.to_categorical(encoded_Y)

# test model for real data

In [20]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_train)
encoded_Y = encoder.transform(y_train)
# One Hot Encode
y_train = np_utils.to_categorical(encoded_Y)

Using TensorFlow backend.
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ASUS\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519

In [21]:
# Label encode Class (Species)
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
encoder = LabelEncoder()
encoder.fit(y_test)
encoded_Y = encoder.transform(y_test)
# One Hot Encode
y_test = np_utils.to_categorical(encoded_Y)

In [22]:
import numpy as np
X_train=np.array(X_train)
y_train=np.array(y_train)
X_test=np.array(X_test)
y_test=np.array(y_test)

# Phase 3: Architecture of RENA Network

In [49]:
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding
import keras

callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc',patience=1,),
keras.callbacks.ModelCheckpoint(filepath='my_model.h5',monitor='val_loss',save_best_only=True,)]
#del model_rnn
model_rnn = Sequential()

model_rnn.add(layers.Embedding(6, 8))
model_rnn.add(layers.SimpleRNN(4))
model_rnn.add(Dense(2, activation='softmax'))
model_rnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc','mae','mse'])
model_rnn.summary()
history_rnn = model_rnn.fit(X_res_train, y_res_train,epochs=2,validation_split=0.2, callbacks=callbacks_list)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (None, None, 8)           48        
_________________________________________________________________
simple_rnn_3 (SimpleRNN)     (None, 4)                 52        
_________________________________________________________________
dense_14 (Dense)             (None, 2)                 10        
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________
Train on 40607796 samples, validate on 10151950 samples
Epoch 1/2
40607796/40607796 [==============================] - 5902s 145us/step - loss: 2.2543e-04 - acc: 1.0000 - mae: -6.2950e-06 - mse: 3.0406e-05 - val_loss: 1.4782e-05 - val_acc: 1.0000 - val_mae: 1.4785e-05 - val_mse: 2.1852e-10
Epoch 2/2
40607796/40607796 [==============================] - 5904s 145us/step - loss: 2.6877

In [52]:
# save model
import h5py
model_rnn.save('large dataset/deep_model_resample_6features_rnn_ec-sc.h5')
model_rnn.save_weights('large dataset/deep_model_resample_6features_rnn_weights_ec-sc.h5')

# test model for real data

In [64]:
from keras.models import load_model
model_rnn = load_model('deep_model_resample_6features_rnn_ec-sc.h5')
model_rnn.load_weights('deep_model_resample_6features_rnn_weights_ec-sc.h5', by_name=True)

In [65]:
results_tr = model_rnn.evaluate(X_train, y_train)
results_tr

25384000/25384000 [==============================] - 1465s 58us/step


[0.0, 1.0, 0.0, 0.0]

In [66]:
%%time
output_tr=model_rnn.predict(X_train)
output_tr

Wall time: 10min 59s


array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [67]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), output_tr.argmax(axis=1)))

[[25379886        0]
 [       0     4114]]


In [68]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), output_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), output_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), output_tr.argmax(axis=1) , average="macro"))

1.0
1.0
1.0


In [69]:
results_te = model_rnn.evaluate(X_test, y_test)
results_te

2820445/2820445 [==============================] - 165s 59us/step


[0.0, 1.0, 0.0, 0.0]

In [70]:
%%time
output_te=model_rnn.predict(X_test)
#print(output_te)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.argmax(axis=1), output_te.argmax(axis=1)))
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_test.argmax(axis=1), output_te.argmax(axis=1) , average="macro"))
print(recall_score(y_test.argmax(axis=1), output_te.argmax(axis=1) , average="macro"))
print(f1_score(y_test.argmax(axis=1), output_te.argmax(axis=1) , average="macro"))

[[2819998       0]
 [      0     447]]
1.0
1.0
1.0
Wall time: 1min 26s


# Train RENA model for real data (without resample)

In [46]:
from keras import Sequential
from keras.layers import Dense,SimpleRNN,Embedding
import keras

callbacks_list = [keras.callbacks.EarlyStopping(monitor='acc',patience=1,),
keras.callbacks.ModelCheckpoint(filepath='my_model.h5',monitor='val_loss',save_best_only=True,)]
#del model_rnn
model_rnn = Sequential()

model_rnn.add(layers.Embedding(6, 8))
model_rnn.add(layers.SimpleRNN(4))
model_rnn.add(Dense(2, activation='softmax'))
model_rnn.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['acc','mae','mse'])
model_rnn.summary()
history_rnn = model_rnn.fit(X_train, y_train,epochs=2,validation_split=0.2, callbacks=callbacks_list)

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, None, 8)           48        
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 4)                 52        
_________________________________________________________________
dense_13 (Dense)             (None, 2)                 10        
Total params: 110
Trainable params: 110
Non-trainable params: 0
_________________________________________________________________
Train on 20307200 samples, validate on 5076800 samples
Epoch 1/2
20307200/20307200 [==============================] - 2932s 144us/step - loss: 8.6748e-04 - acc: 0.9999 - mae: 3.0912e-04 - mse: 1.1521e-04 - val_loss: 5.0945e-05 - val_acc: 1.0000 - val_mae: 5.0020e-05 - val_mse: 1.7195e-06
Epoch 2/2
20307200/20307200 [==============================] - 2949s 145us/step - loss: 3.0630e-

In [48]:
# save model
import h5py
model_rnn.save('large dataset/deep_model_before_resample_6features_rnn_ec-sc.h5')
model_rnn.save_weights('large dataset/deep_model_before_resample_6features_rnn_weights_ec-sc.h5')

# test RNN before resample

In [71]:
from keras.models import load_model
#del model
model_rnn = load_model('large dataset/deep_model_before_resample_6features_rnn_ec-sc.h5')
model_rnn.load_weights('large dataset/deep_model_before_resample_6features_rnn_weights_ec-sc.h5', by_name=True)

In [72]:
results_tr = model_rnn.evaluate(X_train, y_train)
results_tr

25384000/25384000 [==============================] - 1460s 58us/step


[nan, 0.99989253282547, nan, nan]

In [73]:
%%time
output_tr=model_rnn.predict(X_train)
output_tr

Wall time: 11min 30s


array([[nan, nan],
       [nan, nan],
       [nan, nan],
       ...,
       [nan, nan],
       [nan, nan],
       [nan, nan]], dtype=float32)

In [74]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_train.argmax(axis=1), output_tr.argmax(axis=1)))

[[25379886        0]
 [    4114        0]]


In [75]:
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_train.argmax(axis=1), output_tr.argmax(axis=1) , average="macro"))
print(recall_score(y_train.argmax(axis=1), output_tr.argmax(axis=1) , average="macro"))
print(f1_score(y_train.argmax(axis=1), output_tr.argmax(axis=1) , average="macro"))

C:\Users\ASUS\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.4999189647021746
0.5
0.49995947906746147


In [76]:
results_te = model_rnn.evaluate(X_test, y_test)
results_te

2820445/2820445 [==============================] - 162s 57us/step


[nan, 0.9998415112495422, nan, nan]

In [77]:
%%time
output_te=model_rnn.predict(X_test)
#print(output_te)
from sklearn.metrics import confusion_matrix
print(confusion_matrix(y_test.argmax(axis=1), output_te.argmax(axis=1)))
from sklearn.metrics import f1_score, precision_score, recall_score
print(precision_score(y_test.argmax(axis=1), output_te.argmax(axis=1) , average="macro"))
print(recall_score(y_test.argmax(axis=1), output_te.argmax(axis=1) , average="macro"))
print(f1_score(y_test.argmax(axis=1), output_te.argmax(axis=1) , average="macro"))

[[2819998       0]
 [    447       0]]
0.49992075718547957
0.5
0.49996037545277916
Wall time: 1min 21s


# test other classifier without resample

In [78]:
x_tra,y_tra,x_tes,y_tes=X_train,y_train,X_test,y_test

In [79]:
X_train,y_train,X_test,y_test=x_tr,y_tr,x_te,y_te

In [89]:
%%time
# Linear Discriminant Analysis

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
print('Accuracy of LDA classifier on training set: {:.2f}'
     .format(lda.score(X_train, y_train)))
print('Accuracy of LDA classifier on test set: {:.2f}'
     .format(lda.score(X_test, y_test)))

Accuracy of LDA classifier on training set: 1.00
Accuracy of LDA classifier on test set: 1.00
Wall time: 2min 22s


In [90]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix
# Make predictions
preds_tr = lda.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999761660888749
[[25379886        0]
 [     605     3509]]
0.9999880813968492
0.9264705882352942
0.9603115009448575
Wall time: 3min 56s


In [91]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix
# Make predictions
preds = lda.predict(X_test)
print(preds)

# Evaluate accuracy
print(accuracy_score(y_test, preds))

print(confusion_matrix(y_test, preds))
print(precision_score(y_test, preds , average="macro"))
print(recall_score(y_test, preds , average="macro"))
print(f1_score(y_test, preds , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999726993435433
[[2819998       0]
 [     77     370]]
0.9999863478808187
0.9138702460850112
0.952869550836202
Wall time: 25.4 s


In [99]:
import pickle
with open('large dataset/saved_ldamodel_ec-hp.pickle','rb') as f:
    lda_from_pickle=pickle.load(f)
preds=lda_from_pickle.predict(X_test)
# Evaluate accuracy
print(accuracy_score(y_test, preds))

print(confusion_matrix(y_test, preds))
print(precision_score(y_test, preds , average="macro"))
print(recall_score(y_test, preds , average="macro"))
print(f1_score(y_test, preds , average="macro"))

0.9999230617863493
[[2819998       0]
 [    217     230]]
0.9999615277558627
0.7572706935123042
0.8397148842604457


In [100]:
preds_tr = lda_from_pickle.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999238890639773
[[25379886        0]
 [    1932     2182]]
0.9999619412604723
0.7651920272241128
0.8465502202236368


In [ ]:
%%time
# KNN
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
# time: one hour

In [5]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix
# Make predictions
preds_tr = knn.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999808934762054
[[25379886        0]
 [     485     3629]]
0.9999904453721342
0.9410549343704424
0.9686766122826788
Wall time: 20min 4s


In [6]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix,accuracy_score
# Make predictions
preds =  knn.predict(X_test)
print(preds)

# Evaluate accuracy
print(accuracy_score(y_test, preds))

print(confusion_matrix(y_test, preds))
print(precision_score(y_test, preds , average="macro"))
print(recall_score(y_test, preds , average="macro"))
print(f1_score(y_test, preds , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999709265736435
[[2819998       0]
 [     82     365]]
0.9999854614053503
0.9082774049217002
0.9495001197595498
Wall time: 2min 19s


In [7]:
%%time
# SVM
from sklearn.svm import SVC
svm = SVC()
svm.fit(X_train, y_train)
print('Accuracy of SVM classifier on training set: {:.2f}'
     .format(svm.score(X_train, y_train)))
print('Accuracy of SVM classifier on test set: {:.2f}'
     .format(svm.score(X_test, y_test)))

Accuracy of SVM classifier on training set: 1.00
Accuracy of SVM classifier on test set: 1.00
Wall time: 3h 29min 4s


In [8]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds_tr = svm.predict(X_train)
print(preds_tr)

# Evaluate accuracy
print(accuracy_score(y_train, preds_tr))

print(confusion_matrix(y_train, preds_tr))
print(precision_score(y_train, preds_tr , average="macro"))
print(recall_score(y_train, preds_tr , average="macro"))
print(f1_score(y_train, preds_tr , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.99998262685156
[[25379886        0]
 [     441     3673]]
0.999991312168673
0.9464025279533301
0.9716792312360152
Wall time: 42min 41s


In [9]:
%%time
from sklearn.metrics import f1_score, precision_score, recall_score,confusion_matrix,accuracy_score
# Make predictions
preds = svm.predict(X_test)
print(preds)

# Evaluate accuracy
print(accuracy_score(y_test, preds))

print(confusion_matrix(y_test, preds))
print(precision_score(y_test, preds , average="macro"))
print(recall_score(y_test, preds , average="macro"))
print(f1_score(y_test, preds , average="macro"))

['No' 'No' 'No' ... 'No' 'No' 'No']
0.9999840450709019
[[2819998       0]
 [     45     402]]
0.9999920213982553
0.9496644295302014
0.973494243882846
Wall time: 4min 42s
Parser   : 1.64 s
